## SARMAD ALI
## Proposition One: Create and Populate dbo.Customers Table

This query creates a new table called `dbo.Customers` in the `WideWorldImporters` database, then populates it with unique `CustomerID` and `CustomerName` values from the `Sales.Customers` table. Finally, it retrieves the top 5 rows for verification.



In [106]:
USE [WideWorldImporters];
GO

DROP TABLE IF EXISTS dbo.Customers;

CREATE TABLE dbo.Customers
(
  CustomerID      INT          NOT NULL PRIMARY KEY,
  CustomerName    NVARCHAR(100) NOT NULL
);

INSERT INTO dbo.Customers (CustomerID, CustomerName)
SELECT DISTINCT C.CustomerID, C.CustomerName
FROM Sales.Customers AS C


SELECT top(5) * FROM dbo.Customers

Commands completed successfully.

(663 rows affected)

(5 rows affected)

Total execution time: 00:00:00.033

CustomerID,CustomerName
1,Tailspin Toys (Head Office)
2,"Tailspin Toys (Sylvanite, MT)"
3,"Tailspin Toys (Peeples Valley, AZ)"
4,"Tailspin Toys (Medicine Lodge, KS)"
5,"Tailspin Toys (Gasport, NY)"


## Proposition Two: Create and Populate HighValueCustomers Table

This query creates a `HighValueCustomers` table in the `WideWorldImporters` database, which stores customers with a high total order value. The table includes `CustomerID`, `CustomerName`, the total value of all orders (`TotalOrderValue`), and the date of the last order (`LastOrderDate`). Only customers with a `TotalOrderValue` above $10,000 are inserted.


In [115]:
USE WideWorldImporters;

DROP TABLE IF EXISTS dbo.HighValueCustomers;

CREATE TABLE dbo.HighValueCustomers
(
  CustomerID         INT PRIMARY KEY,
  CustomerName       NVARCHAR(100) NOT NULL,
  TotalOrderValue    DECIMAL(18, 2),
  LastOrderDate      DATE
);

INSERT INTO dbo.HighValueCustomers (CustomerID, CustomerName, TotalOrderValue, LastOrderDate)
SELECT C.CustomerID, C.CustomerName, SUM(OL.Quantity * OL.UnitPrice) AS TotalOrderValue, MAX(O.OrderDate) AS LastOrderDate
FROM Sales.Customers AS C
JOIN Sales.Orders AS O ON O.CustomerID = C.CustomerID
JOIN Sales.OrderLines AS OL ON OL.OrderID = O.OrderID
GROUP BY C.CustomerID, C.CustomerName
HAVING SUM(OL.Quantity * OL.UnitPrice) > 10000;  -- Adjust threshold as needed


Select Top (5) * from dbo.HighValueCustomers

(662 rows affected)

(5 rows affected)

Total execution time: 00:00:00.095

CustomerID,CustomerName,TotalOrderValue,LastOrderDate
1,Tailspin Toys (Head Office),315404.00,2016-05-27
2,"Tailspin Toys (Sylvanite, MT)",240817.90,2016-05-14
3,"Tailspin Toys (Peeples Valley, AZ)",317243.45,2016-05-30
4,"Tailspin Toys (Medicine Lodge, KS)",310903.85,2016-04-28
5,"Tailspin Toys (Gasport, NY)",264001.40,2016-05-28


## Proposition Three: Create and Populate RecentCustomerOrders Table

This query creates a `RecentCustomerOrders` table in the `WideWorldImporters` database to store customer information for orders placed in 2015. It includes the `CustomerID`, `CustomerName`, date of the last order (`LastOrderDate`), and the total value of that order (`LastOrderTotal`).

In [119]:
USE WideWorldImporters;

DROP TABLE IF EXISTS dbo.RecentCustomerOrders;

CREATE TABLE dbo.RecentCustomerOrders
(
  CustomerID     INT PRIMARY KEY,
  CustomerName   NVARCHAR(100),
  LastOrderDate  DATE,
  LastOrderTotal DECIMAL(18, 2)
);

INSERT INTO dbo.RecentCustomerOrders (CustomerID, CustomerName, LastOrderDate, LastOrderTotal)
SELECT 
    C.CustomerID, 
    C.CustomerName, 
    MAX(O.OrderDate) AS LastOrderDate,
    SUM(OL.Quantity * OL.UnitPrice) AS LastOrderTotal
FROM Sales.Customers AS C
JOIN Sales.Orders AS O ON C.CustomerID = O.CustomerID
JOIN Sales.OrderLines AS OL ON O.OrderID = OL.OrderID
WHERE YEAR(O.OrderDate) = 2015
GROUP BY C.CustomerID, C.CustomerName;

(657 rows affected)

Total execution time: 00:00:00.088

## Proposition Four: Create and Populate KeySupplierContacts Table

This query creates a `KeySupplierContacts` table in the `WideWorldImporters` database, which stores contact details for key suppliers. The table includes suppliers in specific categories (`SupplierCategoryID` of 2 or 3) and contains fields like `SupplierID`, `SupplierName`, `PaymentDays`, and various contact details.

In [122]:
USE WideWorldImporters;

DROP TABLE IF EXISTS dbo.KeySupplierContacts;

CREATE TABLE dbo.KeySupplierContacts
(
  SupplierID          INT PRIMARY KEY,
  SupplierName        NVARCHAR(100) NOT NULL,
  PaymentDays         INT,
  PrimaryContactID    INT,
  AlternateContactID  INT,
  PhoneNumber         NVARCHAR(20),
  FaxNumber           NVARCHAR(20),
  WebsiteURL          NVARCHAR(255)
);

INSERT INTO dbo.KeySupplierContacts (SupplierID, SupplierName, PaymentDays, PrimaryContactID, AlternateContactID, PhoneNumber, FaxNumber, WebsiteURL)
SELECT 
    SupplierID, 
    SupplierName, 
    PaymentDays, 
    PrimaryContactPersonID, 
    AlternateContactPersonID, 
    PhoneNumber, 
    FaxNumber, 
    WebsiteURL
FROM Purchasing.Suppliers
WHERE SupplierCategoryID IN (2, 3); -- Adjust SupplierCategoryID as needed

SELECT * from dbo.KeySupplierContacts

(7 rows affected)

(7 rows affected)

Total execution time: 00:00:00.030

SupplierID,SupplierName,PaymentDays,PrimaryContactID,AlternateContactID,PhoneNumber,FaxNumber,WebsiteURL
1,A Datum Corporation,14,21,22,(847) 555-0100,(847) 555-0101,http://www.adatum.com
2,"Contoso, Ltd.",7,23,24,(360) 555-0100,(360) 555-0101,http://www.contoso.com
5,Graphic Design Institute,14,29,30,(406) 555-0105,(406) 555-0106,http://www.graphicdesigninstitute.com
8,Lucerne Publishing,30,35,36,(423) 555-0103,(423) 555-0105,http://www.lucernepublishing.com
9,Nod Publishers,7,37,38,(252) 555-0100,(252) 555-0101,http://www.nodpublishers.com
10,Northwind Electric Cars,30,39,40,(201) 555-0105,(201) 555-0104,http://www.northwindelectriccars.com
12,The Phone Company,30,43,44,(218) 555-0105,(218) 555-0105,http://www.thephone-company.com


## Proposition Five: Create and Populate UniqueCities Table

This query creates a `UniqueCities` table in the `WideWorldImporters` database, storing information on city locations. The table categorizes each city by whether it has only suppliers, only customers, or both. This can help in identifying potential areas for expansion.


In [125]:
USE WideWorldImporters;

DROP TABLE IF EXISTS dbo.UniqueCities;

CREATE TABLE dbo.UniqueCities
(
  CityType NVARCHAR(20),
  CityID   INT
);

-- Insert cities where both suppliers and customers are located
INSERT INTO dbo.UniqueCities (CityType, CityID)
SELECT DISTINCT 'Both' AS CityType, DeliveryCityID
FROM Purchasing.Suppliers
WHERE DeliveryCityID IN (SELECT DeliveryCityID FROM Sales.Customers);

-- Insert cities where only suppliers are located
INSERT INTO dbo.UniqueCities (CityType, CityID)
SELECT DISTINCT 'SupplierOnly' AS CityType, DeliveryCityID
FROM Purchasing.Suppliers
WHERE DeliveryCityID NOT IN (SELECT DeliveryCityID FROM Sales.Customers);

-- Insert cities where only customers are located
INSERT INTO dbo.UniqueCities (CityType, CityID)
SELECT DISTINCT 'CustomerOnly' AS CityType, DeliveryCityID
FROM Sales.Customers
WHERE DeliveryCityID NOT IN (SELECT DeliveryCityID FROM Purchasing.Suppliers);

SELECT * FROM dbo.UniqueCities;

(0 rows affected)

(12 rows affected)

(655 rows affected)

(667 rows affected)

Total execution time: 00:00:00.067

CityType,CityID
SupplierOnly,7899
SupplierOnly,10346
SupplierOnly,13870
SupplierOnly,17161
SupplierOnly,17277
SupplierOnly,17346
SupplierOnly,18557
SupplierOnly,18634
SupplierOnly,18656
SupplierOnly,22602


## Proposition Six: Create and Populate TopCustomerProducts Table

This query creates a `TopCustomerProducts` table in the `WideWorldImporters` database, recording the most frequently ordered product for each customer. The table includes the `CustomerID`, `CustomerName`, `StockItemID`, `ProductName`, and `TotalQuantity` of the top product.


In [128]:
USE WideWorldImporters;

DROP TABLE IF EXISTS dbo.TopCustomerProducts;

CREATE TABLE dbo.TopCustomerProducts
(
  CustomerID      INT,
  CustomerName    NVARCHAR(100),
  StockItemID     INT,
  ProductName     NVARCHAR(100),
  TotalQuantity   INT,
  PRIMARY KEY (CustomerID, StockItemID)
);

WITH CustomerProductTotals AS (
    SELECT 
        C.CustomerID,
        C.CustomerName,
        OL.StockItemID,
        SI.StockItemName AS ProductName,
        SUM(OL.Quantity) AS TotalQuantity,
        RANK() OVER (PARTITION BY C.CustomerID ORDER BY SUM(OL.Quantity) DESC) AS ProductRank
    FROM Sales.Customers AS C
    JOIN Sales.Orders AS O ON C.CustomerID = O.CustomerID
    JOIN Sales.OrderLines AS OL ON O.OrderID = OL.OrderID
    JOIN Warehouse.StockItems AS SI ON OL.StockItemID = SI.StockItemID
    GROUP BY C.CustomerID, C.CustomerName, OL.StockItemID, SI.StockItemName
)

-- Select only the top product for each customer based on the highest quantity ordered
INSERT INTO dbo.TopCustomerProducts (CustomerID, CustomerName, StockItemID, ProductName, TotalQuantity)
SELECT CustomerID, CustomerName, StockItemID, ProductName, TotalQuantity
FROM CustomerProductTotals
WHERE ProductRank = 1;

SELECT * FROM dbo.TopCustomerProducts;

(694 rows affected)

(694 rows affected)

Total execution time: 00:00:00.156

CustomerID,CustomerName,StockItemID,ProductName,TotalQuantity
1,Tailspin Toys (Head Office),194,Black and orange glass with care despatch tape 48mmx100m,744
2,"Tailspin Toys (Sylvanite, MT)",177,Shipping carton (Brown) 413x285x187mm,1025
3,"Tailspin Toys (Peeples Valley, AZ)",189,Clear packaging tape 48mmx75m,624
4,"Tailspin Toys (Medicine Lodge, KS)",195,Black and orange handle with care despatch tape 48mmx75m,816
5,"Tailspin Toys (Gasport, NY)",188,3 kg Courier post bag (White) 300x190x95mm,725
6,"Tailspin Toys (Jessie, ND)",178,Shipping carton (Brown) 500x310x310mm,550
7,"Tailspin Toys (Frankewing, TN)",202,Red and white urgent heavy despatch tape 48mmx100m,1032
8,"Tailspin Toys (Bow Mar, CO)",187,Express post box 5kg (White) 350x280x130mm,650
9,"Tailspin Toys (Netcong, NJ)",192,Black and orange fragile despatch tape 48mmx100m,828
10,"Tailspin Toys (Wimbledon, ND)",197,Black and orange this way up despatch tape 48mmx75m,864


## Proposition Seven: Create and Populate HighValueCustomers Table Using Temporary Table

This query creates a `HighValueCustomers` table in the `WideWorldImporters` database to store customers with a high total spend (above $10,000). A temporary table (`#CustomerSpend`) is used to calculate each customer’s total spend before filtering and inserting into the final table.



In [145]:
USE WideWorldImporters;

-- Step 1: Create a temporary table to store total spend per customer
DROP TABLE IF EXISTS #CustomerSpend;

CREATE TABLE #CustomerSpend
(
  CustomerID      INT,
  CustomerName    NVARCHAR(100),
  TotalSpend      DECIMAL(18, 2)
);

-- Populate the temporary table with total spend for each customer
INSERT INTO #CustomerSpend (CustomerID, CustomerName, TotalSpend)
SELECT 
    C.CustomerID, 
    C.CustomerName, 
    SUM(OL.Quantity * OL.UnitPrice) AS TotalSpend
FROM Sales.Customers AS C
JOIN Sales.Orders AS O ON C.CustomerID = O.CustomerID
JOIN Sales.OrderLines AS OL ON O.OrderID = OL.OrderID
GROUP BY C.CustomerID, C.CustomerName;

-- Step 2: Insert into HighValueCustomers from #CustomerSpend where TotalSpend > 10000
DROP TABLE IF EXISTS dbo.HighValueCustomers;

CREATE TABLE dbo.HighValueCustomers
(
  CustomerID      INT PRIMARY KEY,
  CustomerName    NVARCHAR(100),
  TotalSpend      DECIMAL(18, 2)
);

INSERT INTO dbo.HighValueCustomers (CustomerID, CustomerName, TotalSpend)
SELECT CustomerID, CustomerName, TotalSpend
FROM #CustomerSpend
WHERE TotalSpend > 10000;

-- Clean up temporary table
DROP TABLE #CustomerSpend;

-- Display results
SELECT * FROM dbo.HighValueCustomers;

(663 rows affected)

(662 rows affected)

(662 rows affected)

Total execution time: 00:00:00.074

CustomerID,CustomerName,TotalSpend
1,Tailspin Toys (Head Office),315404.00
2,"Tailspin Toys (Sylvanite, MT)",240817.90
3,"Tailspin Toys (Peeples Valley, AZ)",317243.45
4,"Tailspin Toys (Medicine Lodge, KS)",310903.85
5,"Tailspin Toys (Gasport, NY)",264001.40
6,"Tailspin Toys (Jessie, ND)",266529.60
7,"Tailspin Toys (Frankewing, TN)",332695.75
8,"Tailspin Toys (Bow Mar, CO)",236985.40
9,"Tailspin Toys (Netcong, NJ)",299916.75
10,"Tailspin Toys (Wimbledon, ND)",319869.65


## Proposition Eight: Create and Populate FirstTimeCustomers2015 Table

This query creates a `FirstTimeCustomers2015` table in the `WideWorldImporters` database to record customers who placed their first order in 2015, along with their total spend for that year. The table includes `CustomerID`, `CustomerName`, the date of their first order (`FirstOrderDate`), and their total spend in 2015 (`TotalSpend2015`).


In [137]:
USE [WideWorldImporters];
GO

USE WideWorldImporters;

DROP TABLE IF EXISTS dbo.FirstTimeCustomers2015;

CREATE TABLE dbo.FirstTimeCustomers2015
(
  CustomerID      INT PRIMARY KEY,
  CustomerName    NVARCHAR(100),
  FirstOrderDate  DATE,
  TotalSpend2015  DECIMAL(18, 2)
);

-- Insert customers who placed their first order in 2015 along with their total spend for that year
INSERT INTO dbo.FirstTimeCustomers2015 (CustomerID, CustomerName, FirstOrderDate, TotalSpend2015)
SELECT 
    C.CustomerID, 
    C.CustomerName,
    MIN(O.OrderDate) AS FirstOrderDate,
    SUM(OL.Quantity * OL.UnitPrice) AS TotalSpend2015
FROM Sales.Customers AS C
JOIN Sales.Orders AS O ON C.CustomerID = O.CustomerID
JOIN Sales.OrderLines AS OL ON O.OrderID = OL.OrderID
WHERE YEAR(O.OrderDate) = 2015
GROUP BY C.CustomerID, C.CustomerName
HAVING MIN(O.OrderDate) BETWEEN '2015-01-01' AND '2015-12-31';

SELECT * FROM dbo.FirstTimeCustomers2015;

Commands completed successfully.

(657 rows affected)

(657 rows affected)

Total execution time: 00:00:00.130

CustomerID,CustomerName,FirstOrderDate,TotalSpend2015
1,Tailspin Toys (Head Office),2015-01-05,117697.80
2,"Tailspin Toys (Sylvanite, MT)",2015-01-02,89795.45
3,"Tailspin Toys (Peeples Valley, AZ)",2015-01-14,78945.70
4,"Tailspin Toys (Medicine Lodge, KS)",2015-01-06,82504.85
5,"Tailspin Toys (Gasport, NY)",2015-01-31,74840.20
6,"Tailspin Toys (Jessie, ND)",2015-01-12,60149.20
7,"Tailspin Toys (Frankewing, TN)",2015-01-14,80081.55
8,"Tailspin Toys (Bow Mar, CO)",2015-01-01,70632.90
9,"Tailspin Toys (Netcong, NJ)",2015-01-05,102902.15
10,"Tailspin Toys (Wimbledon, ND)",2015-01-01,108327.85


## Proposition Nine: Create and Populate BestSellingProducts Table

This query creates a `BestSellingProducts` table in the `WideWorldImporters` database to store the most popular products based on total quantity ordered. The table includes `StockItemID`, `ProductName`, and `TotalQuantity` for the best-selling products.

In [140]:
USE WideWorldImporters;

DROP TABLE IF EXISTS dbo.BestSellingProducts;

CREATE TABLE dbo.BestSellingProducts
(
  StockItemID      INT PRIMARY KEY,
  ProductName      NVARCHAR(100),
  TotalQuantity    INT
);

-- Insert the top-selling products based on total quantity ordered
INSERT INTO dbo.BestSellingProducts (StockItemID, ProductName, TotalQuantity)
SELECT 
    SI.StockItemID, 
    SI.StockItemName AS ProductName,
    SUM(OL.Quantity) AS TotalQuantity
FROM Sales.OrderLines AS OL
JOIN Warehouse.StockItems AS SI ON OL.StockItemID = SI.StockItemID
GROUP BY SI.StockItemID, SI.StockItemName
HAVING SUM(OL.Quantity) = (
    SELECT MAX(TotalQuantity)
    FROM (
        SELECT SUM(Quantity) AS TotalQuantity
        FROM Sales.OrderLines
        GROUP BY StockItemID
    ) AS ProductQuantities
);

SELECT * FROM dbo.BestSellingProducts;

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.055

StockItemID,ProductName,TotalQuantity
191,Black and orange fragile despatch tape 48mmx75m,207324


## Proposition Ten: Create and Populate TopCustomersByOrders Table

This query creates a `TopCustomersByOrders` table in the `WideWorldImporters` database, recording the customers with the highest number of orders. The table includes the `CustomerID`, `CustomerName`, and `OrderCount` for each customer.

In [146]:
USE WideWorldImporters;

DROP TABLE IF EXISTS dbo.TopCustomersByOrders;

CREATE TABLE dbo.TopCustomersByOrders
(
  CustomerID      INT PRIMARY KEY,
  CustomerName    NVARCHAR(100),
  OrderCount      INT
);

-- Step 1: Calculate order count for each customer
INSERT INTO dbo.TopCustomersByOrders (CustomerID, CustomerName, OrderCount)
SELECT 
    C.CustomerID, 
    C.CustomerName, 
    COUNT(O.OrderID) AS OrderCount
FROM Sales.Customers AS C
JOIN Sales.Orders AS O ON C.CustomerID = O.CustomerID
GROUP BY C.CustomerID, C.CustomerName
HAVING COUNT(O.OrderID) = (
    SELECT MAX(OrderCount)
    FROM (
        SELECT COUNT(OrderID) AS OrderCount
        FROM Sales.Orders
        GROUP BY CustomerID
    ) AS CustomerOrderCounts
);

-- Display results
SELECT * FROM dbo.TopCustomersByOrders;

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.083

CustomerID,CustomerName,OrderCount
90,"Tailspin Toys (Tolna, ND)",150
